Disaster Management:
This code tries to access Twitter API

In [2]:
# Libraries required
# https://towardsdatascience.com/an-extensive-guide-to-collecting-tweets-from-twitter-api-v2-for-academic-research-using-python-3-518fcb71df2a

In [3]:
# For sending GET requests from the API
import requests
# For saving access tokens and for file management when creating and adding to the dataset
import os
# For dealing with json responses we receive from the API
import json
# For displaying the data after
import pandas as pd
# For saving the response data in CSV format
import csv
# For parsing the dates received from twitter in readable formats
import datetime
import dateutil.parser
import unicodedata
#To add wait time between requests
import time

In [19]:
token = ##enter twitter token id
os.environ['TOKEN'] = token

In [20]:
def auth():
    return os.getenv('TOKEN')

In [21]:
def create_headers(bearer_token):
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    return headers

In [22]:
def create_url(keyword, start_date, end_date, max_results = 10):
    
    search_url = "https://api.twitter.com/2/tweets/search/all" #Change to the endpoint you want to collect data from

    #change params based on the endpoint you are using
    query_params = {'query': keyword,
                    'start_time': start_date,
                    'end_time': end_date,
                    'max_results': max_results,
                    'expansions': 'author_id,in_reply_to_user_id,geo.place_id',
                    'tweet.fields': 'id,text,author_id,in_reply_to_user_id,geo,conversation_id,created_at,lang,public_metrics,referenced_tweets,reply_settings,source',
                    'user.fields': 'id,name,username,created_at,description,public_metrics,verified',
                    'place.fields': 'full_name,id,country,country_code,geo,name,place_type',
                    'next_token': {}}
    return (search_url, query_params)

In [23]:
def connect_to_endpoint(url, headers, params, next_token = None):
    params['next_token'] = next_token   #params object received from create_url function
    response = requests.request("GET", url, headers = headers, params = params)
    print("Endpoint Response Code: " + str(response.status_code))
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()

In [24]:
#Inputs for the request
bearer_token = auth()
headers = create_headers(bearer_token)
keyword = "Hurricane" or "Florence"
start_time = "2021-03-01T00:00:00.000Z"
end_time = "2021-03-31T00:00:00.000Z"
max_results = 15

In [25]:
url = create_url(keyword, start_time,end_time, max_results)
json_response = connect_to_endpoint(url[0], headers, url[1])

Endpoint Response Code: 200


In [26]:
#print(json.dumps(json_response, indent=4, sort_keys=True))

In [27]:
# Saving the output json file to CSV

# Create file
csvFile = open("data.csv", "a", newline="", encoding='utf-8')
csvWriter = csv.writer(csvFile)

#Create headers for the data you want to save, in this example, we only want save these columns in our dataset
csvWriter.writerow(['author id', 'created_at', 'geo', 'id','lang', 'like_count', 'quote_count', 'reply_count','retweet_count','source','tweet'])
csvFile.close()

In [28]:
def append_to_csv(json_response, fileName):

    #A counter variable
    counter = 0

    #Open OR create the target CSV file
    csvFile = open(fileName, "a", newline="", encoding='utf-8')
    csvWriter = csv.writer(csvFile)

    #Loop through each tweet
    for tweet in json_response['data']:
        
        # We will create a variable for each since some of the keys might not exist for some tweets
        # So we will account for that

        # 1. Author ID
        author_id = tweet['author_id']

        # 2. Time created
        created_at = dateutil.parser.parse(tweet['created_at'])

        # 3. Geolocation
        if ('geo' in tweet):   
            geo = tweet['geo']['place_id']
        else:
            geo = " "

        # 4. Tweet ID
        tweet_id = tweet['id']

        # 5. Language
        lang = tweet['lang']

        # 6. Tweet metrics
        retweet_count = tweet['public_metrics']['retweet_count']
        reply_count = tweet['public_metrics']['reply_count']
        like_count = tweet['public_metrics']['like_count']
        quote_count = tweet['public_metrics']['quote_count']

        # 7. source
        source = tweet['source']

        # 8. Tweet text
        text = tweet['text']
        
        # Assemble all data in a list
        res = [author_id, created_at, geo, tweet_id, lang, like_count, quote_count, reply_count, retweet_count, source, text]
        
        # Append the result to the CSV file
        csvWriter.writerow(res)
        counter += 1

    # When done, close the CSV file
    csvFile.close()

    # Print the number of tweets for this iteration
    print("# of Tweets added from this response: ", counter) 


In [29]:
append_to_csv(json_response, "data.csv")

# of Tweets added from this response:  15


## Hurricane Florence (2018)

In [ ]:

#Inputs for tweets
bearer_token = auth()
headers = create_headers(bearer_token)
keyword = "Hurricane" or "#HurricaneFlorence"
start_list =    ['2018-09-13T00:00:00.000Z']

end_list =      ['2018-09-16T00:00:00.000Z']
#max_results = 500

#Total number of tweets we collected from the loop
total_tweets = 0

# Create file
csvFile = open("data_florence.csv", "a", newline="", encoding='utf-8')
csvWriter = csv.writer(csvFile)

#Create headers for the data you want to save, in this example, we only want save these columns in our dataset
csvWriter.writerow(['author id', 'created_at', 'geo', 'id','lang', 'like_count', 'quote_count', 'reply_count','retweet_count','source','tweet'])
csvFile.close()

for i in range(0,len(start_list)):

    # Inputs
    count = 0 # Counting tweets per time period
    #max_count = 10000 # Max tweets per time period
    flag = True
    next_token = None
    
    # Check if flag is true
    while flag:
        # Check if max_count reached
        #if count >= max_count:
        #    break
        print("-------------------")
        print("Token: ", next_token)
        url = create_url(keyword, start_list[i],end_list[i], max_results)
        json_response = connect_to_endpoint(url[0], headers, url[1], next_token)
        result_count = json_response['meta']['result_count']
        print(result_count)

        if 'next_token' in json_response['meta']:
            # Save the token to use for next call
            next_token = json_response['meta']['next_token']
            print("Next Token: ", next_token)
            if result_count is not None and result_count > 0 and next_token is not None:
                print("Start Date: ", start_list[i])
                append_to_csv(json_response, "data_florence.csv")
                count += result_count
                total_tweets += result_count
                print("Total # of Tweets added: ", total_tweets)
                #print("data.csv")
                print("-------------------")
                time.sleep(5)                
        # If no next token exists
        else:
            if result_count is not None and result_count > 0:
                print("-------------------")
                print("Start Date: ", start_list[i])
                append_to_csv(json_response, "data_florence.csv")
                #print('data.csv')
                count += result_count
                total_tweets += result_count
                print("Total # of Tweets added: ", total_tweets)
                print("-------------------")
                time.sleep(5)
            
            #Since this is the final request, turn flag to false to move to the next time period.
            flag = False
            next_token = None
        time.sleep(5)
print("Total number of results: ", total_tweets)

-------------------
Token:  None
Endpoint Response Code: 200
490
Next Token:  b26v89c19zqg8o3fn0ml4fl9gmrnx0sv86x6lpg2q63ct
Start Date:  2018-09-13T00:00:00.000Z
# of Tweets added from this response:  490
Total # of Tweets added:  490
-------------------
-------------------
Token:  b26v89c19zqg8o3fn0ml4fl9gmrnx0sv86x6lpg2q63ct
Endpoint Response Code: 200
494
Next Token:  b26v89c19zqg8o3fn0ml4fl9f4kn8o6v4jkjkoh6n3zwd
Start Date:  2018-09-13T00:00:00.000Z
# of Tweets added from this response:  494
Total # of Tweets added:  984
-------------------
-------------------
Token:  b26v89c19zqg8o3fn0ml4fl9f4kn8o6v4jkjkoh6n3zwd
Endpoint Response Code: 200
490
Next Token:  b26v89c19zqg8o3fn0ml4fl9f32p1prlqvo4kdfcpor5p
Start Date:  2018-09-13T00:00:00.000Z
# of Tweets added from this response:  490
Total # of Tweets added:  1474
-------------------
-------------------
Token:  b26v89c19zqg8o3fn0ml4fl9f32p1prlqvo4kdfcpor5p
Endpoint Response Code: 200
492
Next Token:  b26v89c19zqg8o3fn0ml4fl9dkvpg3tev

In [ ]:
open("data_florence.csv").read()

'author id,created_at,geo,id,lang,like_count,quote_count,reply_count,retweet_count,source,tweet\n834672494,2018-09-15 23:59:59+00:00, ,1041114436134739968,en,0,0,0,43,Twitter for iPhone,"RT @ABC: Millions of residents have evacuated from the path of Hurricane #Florence, now a tropical storm, leaving thousands of homes and bu…"\n226821055,2018-09-15 23:59:58+00:00, ,1041114432988803072,en,0,0,0,418,Twitter for Android,RT @StormchaserUKEU: Anyone know of someone who washes a car in a hurricane??? Especially Madame #Florence 😂 #HurricaneFlorence #USA https:…\n1681257270,2018-09-15 23:59:57+00:00, ,1041114430015254528,en,0,0,0,264,Twitter for Android,"RT @WXII: 🦌 We often forget that wildlife must navigate during natural disasters, too. https://t.co/ye0s0qRcIu\n\n#deer #florence #TropicalSt…"\n2208132859,2018-09-15 23:59:57+00:00, ,1041114429902004224,en,0,0,1,1,Twitter for iPhone,"@lantarrykkozar @realDonaldTrump Dear Puerto Rico,\nI did not create the Hurricane. Your Govt failed to distr

In [ ]:
#import re
#x = re.split(',|\n\n|\n',open('data.csv').read())
#x

In [ ]:
df_florence = pd.read_csv(open('data_florence.csv'))
df_florence.head()

,author id,created_at,geo,id,lang,like_count,quote_count,reply_count,retweet_count,source,tweet
0,834672494,2018-09-15 23:59:59+00:00,,1041114436134739968,en,0,0,0,43,Twitter for iPhone,RT @ABC: Millions of residents have evacuated ...
1,226821055,2018-09-15 23:59:58+00:00,,1041114432988803072,en,0,0,0,418,Twitter for Android,RT @StormchaserUKEU: Anyone know of someone wh...
2,1681257270,2018-09-15 23:59:57+00:00,,1041114430015254528,en,0,0,0,264,Twitter for Android,RT @WXII: 🦌 We often forget that wildlife must...
3,2208132859,2018-09-15 23:59:57+00:00,,1041114429902004224,en,0,0,1,1,Twitter for iPhone,@lantarrykkozar @realDonaldTrump Dear Puerto R...
4,1040400979122941955,2018-09-15 23:59:57+00:00,,1041114429801332737,en,0,0,0,0,Twitter for Android,How come when a hurricane hits a Republican st...


In [30]:
#mounting google drive
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
#saving the file to google drive
df_florence.to_csv('/content/drive/My Drive/florence_data.csv')
!cp data.csv "drive/My Drive/"

In [ ]:
#contents of drive (to verify that the data is uploaded)
!ls "/content/drive/MyDrive"

'140801_BITS _Pilani-GOA _Campus_VirtualBaja2015_presentation.pdf'
'140801_BITS _Pilani-GOA _Campus_VirtualBaja2015_presentation.ppt'
'14298 KRIEGER VEC.pdf'
'2015_Virtual Baja SAE India Guidelines.pdf'
'ANSYS CFD PARTICIPATION CERTIFICATE.pdf'
'Basic and Applied Thermodynamics - PK Nag.pdf'
 Binani
'Binani Experience Letter.pdf'
'Birth Certificate.pdf'
 BITS
'BPGC AP 2012 (Responses).gsheet'
'Cam pics'
 CAT
'Colab Notebooks'
 Colleges.gsheet
'Copy of PS2 Sem 1 14-15 Responses.gform'
'Copy of PS2 Sem 1 14-15 (Responses).gsheet'
 data.csv
'Data Diggers Presentation FINAL DRAFT.pptx'
 DSC_0019.jpg
 DSC_0020.jpg
 DSC_0021.jpg
 DSC_0022.jpg
 DSC_0023.jpg
 DSC_0024.jpg
 DSC_0025.jpg
 DSC_0026.jpg
 DSC_0027.jpg
 DSC_0028.jpg
 DSC_0029.jpg
 DSC_0030.jpg
 DSC_0031.jpg
 DSC_0032.jpg
 DSC_0033.jpg
 DSC_0034.jpg
 DSC_0035.jpg
 DSC_0036.jpg
 DSC_0037.jpg
 DSC_0038.jpg
 DSC_0039.jpg
 DSC_0040.jpg
 DSC_0041.jpg
 DSC_0042.jpg
 DSC_0043.jpg
 DSC_0044.jpg
 DSC_0045.jpg
 DSC_0046.jpg
 DSC_0047.jpg
 DSC_

## Hurricane Harvey (2017)

In [ ]:

#Inputs for tweets
bearer_token = auth()
headers = create_headers(bearer_token)
keyword = "Hurricane" or "#HurricaneHarvey"
start_list =    ['2017-08-24T00:00:00.000Z']

end_list =      ['2017-08-31T00:00:00.000Z']
max_results = 500

#Total number of tweets we collected from the loop
total_tweets = 0

# Create file
csvFile = open("data_harvey.csv", "a", newline="", encoding='utf-8')
csvWriter = csv.writer(csvFile)

#Create headers for the data you want to save, in this example, we only want save these columns in our dataset
csvWriter.writerow(['author id', 'created_at', 'geo', 'id','lang', 'like_count', 'quote_count', 'reply_count','retweet_count','source','tweet'])
csvFile.close()

for i in range(0,len(start_list)):

    # Inputs
    count = 0 # Counting tweets per time period
    #max_count = 10000 # Max tweets per time period
    flag = True
    next_token = None
    
    # Check if flag is true
    while flag:
        # Check if max_count reached
        #if count >= max_count:
        #    break
        print("-------------------")
        print("Token: ", next_token)
        url = create_url(keyword, start_list[i],end_list[i], max_results)
        json_response = connect_to_endpoint(url[0], headers, url[1], next_token)
        result_count = json_response['meta']['result_count']

        if 'next_token' in json_response['meta']:
            # Save the token to use for next call
            next_token = json_response['meta']['next_token']
            print("Next Token: ", next_token)
            if result_count is not None and result_count > 0 and next_token is not None:
                print("Start Date: ", start_list[i])
                append_to_csv(json_response, "data_harvey.csv")
                count += result_count
                total_tweets += result_count
                print("Total # of Tweets added: ", total_tweets)
                #print("data.csv")
                print("-------------------")
                time.sleep(5)                
        # If no next token exists
        else:
            if result_count is not None and result_count > 0:
                print("-------------------")
                print("Start Date: ", start_list[i])
                append_to_csv(json_response, "data_harvey.csv")
                #print('data_harvey.csv')
                count += result_count
                total_tweets += result_count
                print("Total # of Tweets added: ", total_tweets)
                print("-------------------")
                time.sleep(5)
            
            #Since this is the final request, turn flag to false to move to the next time period.
            flag = False
            next_token = None
        time.sleep(5)
print("Total number of results: ", total_tweets)

Streaming output truncated to the last 5000 lines.
-------------------
Token:  1jzu9lk96gu5npw448fps8pwbgv476bxocgvrp529119
Endpoint Response Code: 200
Next Token:  1jzu9lk96gu5npw448fps8pwb1etm8pvjaq72rccf62l
Start Date:  2017-08-24T00:00:00.000Z
# of Tweets added from this response:  476
Total # of Tweets added:  346141
-------------------
-------------------
Token:  1jzu9lk96gu5npw448fps8pwb1etm8pvjaq72rccf62l
Endpoint Response Code: 200
Next Token:  1jzu9lk96gu5npw448fps8pwaehj29l87q4a9xji1y0t
Start Date:  2017-08-24T00:00:00.000Z
# of Tweets added from this response:  488
Total # of Tweets added:  346629
-------------------
-------------------
Token:  1jzu9lk96gu5npw448fps8pwaehj29l87q4a9xji1y0t
Endpoint Response Code: 200
Next Token:  1jzu9lk96gu5npw448fps8pw9ywwxn7sta31qui80dj1
Start Date:  2017-08-24T00:00:00.000Z
# of Tweets added from this response:  492
Total # of Tweets added:  347121
-------------------
-------------------
Token:  1jzu9lk96gu5npw448fps8pw9ywwxn7sta31qui80d

In [5]:
df_harvey = pd.read_csv(open("data_harvey.csv"))
df_harvey.head()

#saving the file to google drive
df_harvey.to_csv('/content/drive/My Drive/harvey_data.csv')
!cp data.csv "drive/My Drive/"


FileNotFoundError: ignored

## Missouri Flooding (2019)

In [ ]:

#Inputs for tweets
bearer_token = auth()
headers = create_headers(bearer_token)
keyword = "Missouri river" or "river Missouri" or "flooding" or "north central flooding" or "#missouriflooding"
start_list =    ['2019-03-01T00:00:00.000Z']

end_list =      ['2019-03-31T00:00:00.000Z']
max_results = 500

#Total number of tweets we collected from the loop
total_tweets = 0

# Create file
csvFile = open("data_missouri.csv", "a", newline="", encoding='utf-8')
csvWriter = csv.writer(csvFile)

#Create headers for the data you want to save, in this example, we only want save these columns in our dataset
csvWriter.writerow(['author id', 'created_at', 'geo', 'id','lang', 'like_count', 'quote_count', 'reply_count','retweet_count','source','tweet'])
csvFile.close()

for i in range(0,len(start_list)):

    # Inputs
    count = 0 # Counting tweets per time period
    #max_count = 10000 # Max tweets per time period
    flag = True
    next_token = None
    
    # Check if flag is true
    while flag:
        # Check if max_count reached
        #if count >= max_count:
        #    break
        print("-------------------")
        print("Token: ", next_token)
        url = create_url(keyword, start_list[i],end_list[i], max_results)
        json_response = connect_to_endpoint(url[0], headers, url[1], next_token)
        result_count = json_response['meta']['result_count']

        if 'next_token' in json_response['meta']:
            # Save the token to use for next call
            next_token = json_response['meta']['next_token']
            print("Next Token: ", next_token)
            if result_count is not None and result_count > 0 and next_token is not None:
                print("Start Date: ", start_list[i])
                append_to_csv(json_response, "data_missouri.csv")
                count += result_count
                total_tweets += result_count
                print("Total # of Tweets added: ", total_tweets)
                #print("data_missouri.csv")
                print("-------------------")
                time.sleep(5)                
        # If no next token exists
        else:
            if result_count is not None and result_count > 0:
                print("-------------------")
                print("Start Date: ", start_list[i])
                append_to_csv(json_response, "data_missouri.csv")
                #print('data_missouri.csv')
                count += result_count
                total_tweets += result_count
                print("Total # of Tweets added: ", total_tweets)
                print("-------------------")
                time.sleep(5)
            
            #Since this is the final request, turn flag to false to move to the next time period.
            flag = False
            next_token = None
        time.sleep(5)
print("Total number of results: ", total_tweets)

-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fnlo04uo4hrkgda7idl7m6uya4dke5
Start Date:  2019-03-01T00:00:00.000Z
# of Tweets added from this response:  486
Total # of Tweets added:  486
-------------------
-------------------
Token:  b26v89c19zqg8o3fnlo04uo4hrkgda7idl7m6uya4dke5
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fnlo04o9xvorxkh9yr8e1woavkcuil
Start Date:  2019-03-01T00:00:00.000Z
# of Tweets added from this response:  498
Total # of Tweets added:  984
-------------------
-------------------
Token:  b26v89c19zqg8o3fnlo04o9xvorxkh9yr8e1woavkcuil
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fnlnzpvi5o8gfjgbivu9mp08ybwakd
Start Date:  2019-03-01T00:00:00.000Z
# of Tweets added from this response:  495
Total # of Tweets added:  1479
-------------------
-------------------
Token:  b26v89c19zqg8o3fnlnzpvi5o8gfjgbivu9mp08ybwakd
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fnlnzpr60cqrmue8gu3zsaj34ggcql
Sta

In [ ]:
df_missouri = pd.read_csv(open('data_missouri.csv'))
df_missouri.head()

#saving the file to google drive
df_missouri.to_csv('/content/drive/My Drive/missouri_data.csv')
!cp data.csv "drive/My Drive/"


## Hurricane Laura (2020)

In [ ]:
#Inputs for tweets
bearer_token = auth()
headers = create_headers(bearer_token)
keyword = "hurricane" or "#hurricanelaura"
start_list =    ['2020-08-27T00:00:00.000Z']

end_list =      ['2020-08-28T00:00:00.000Z']
max_results = 500

#Total number of tweets we collected from the loop
total_tweets = 0

# Create file
csvFile = open("data_laura.csv", "a", newline="", encoding='utf-8')
csvWriter = csv.writer(csvFile)

#Create headers for the data you want to save, in this example, we only want save these columns in our dataset
csvWriter.writerow(['author id', 'created_at', 'geo', 'id','lang', 'like_count', 'quote_count', 'reply_count','retweet_count','source','tweet'])
csvFile.close()

for i in range(0,len(start_list)):

    # Inputs
    count = 0 # Counting tweets per time period
    #max_count = 10000 # Max tweets per time period
    flag = True
    next_token = None
    
    # Check if flag is true
    while flag:
        # Check if max_count reached
        #if count >= max_count:
        #    break
        print("-------------------")
        print("Token: ", next_token)
        url = create_url(keyword, start_list[i],end_list[i], max_results)
        json_response = connect_to_endpoint(url[0], headers, url[1], next_token)
        result_count = json_response['meta']['result_count']

        if 'next_token' in json_response['meta']:
            # Save the token to use for next call
            next_token = json_response['meta']['next_token']
            print("Next Token: ", next_token)
            if result_count is not None and result_count > 0 and next_token is not None:
                print("Start Date: ", start_list[i])
                append_to_csv(json_response, "data_laura.csv")
                count += result_count
                total_tweets += result_count
                print("Total # of Tweets added: ", total_tweets)
                #print("data_missouri.csv")
                print("-------------------")
                time.sleep(5)                
        # If no next token exists
        else:
            if result_count is not None and result_count > 0:
                print("-------------------")
                print("Start Date: ", start_list[i])
                append_to_csv(json_response, "data_laura.csv")
                #print('data_missouri.csv')
                count += result_count
                total_tweets += result_count
                print("Total # of Tweets added: ", total_tweets)
                print("-------------------")
                time.sleep(5)
            
            #Since this is the final request, turn flag to false to move to the next time period.
            flag = False
            next_token = None
        time.sleep(5)
print("Total number of results: ", total_tweets)

-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fo7mg0pnt9x5b6a1dvwnvjts054a65
Start Date:  2020-08-27T00:00:00.000Z
# of Tweets added from this response:  481
Total # of Tweets added:  481
-------------------
-------------------
Token:  b26v89c19zqg8o3fo7mg0pnt9x5b6a1dvwnvjts054a65
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fo7mg0pnt8flgdlruab0jnsqwit6kd
Start Date:  2020-08-27T00:00:00.000Z
# of Tweets added from this response:  487
Total # of Tweets added:  968
-------------------
-------------------
Token:  b26v89c19zqg8o3fo7mg0pnt8flgdlruab0jnsqwit6kd
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fo7mg0pnt8ej3wkp4l01gdd6ur0j99
Start Date:  2020-08-27T00:00:00.000Z
# of Tweets added from this response:  483
Total # of Tweets added:  1451
-------------------
-------------------
Token:  b26v89c19zqg8o3fo7mg0pnt8ej3wkp4l01gdd6ur0j99
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fo7mg0pnt6x6koz9ugwu3i7grk99bx
Sta

In [ ]:
df_laura = pd.read_csv(open('data_laura.csv'))
df_laura.head()

#saving the file to google drive
df_laura.to_csv('/content/drive/My Drive/laura_data.csv')
!cp data.csv "drive/My Drive/"

## Texas Winter Storm (2021)

In [ ]:
#Inputs for tweets
bearer_token = auth()
headers = create_headers(bearer_token)
keyword = "Texas" or "winter storm" or "power crisis" or "#texasstorm"
start_list =    ['2021-02-10T00:00:00.000Z']

end_list =      ['2021-02-27T00:00:00.000Z']
max_results = 500

#Total number of tweets we collected from the loop
total_tweets = 0

# Create file
csvFile = open("data_texas.csv", "a", newline="", encoding='utf-8')
csvWriter = csv.writer(csvFile)

#Create headers for the data you want to save, in this example, we only want save these columns in our dataset
csvWriter.writerow(['author id', 'created_at', 'geo', 'id','lang', 'like_count', 'quote_count', 'reply_count','retweet_count','source','tweet'])
csvFile.close()

for i in range(0,len(start_list)):

    # Inputs
    count = 0 # Counting tweets per time period
    #max_count = 10000 # Max tweets per time period
    flag = True
    next_token = None
    
    # Check if flag is true
    while flag:
        # Check if max_count reached
        #if count >= max_count:
        #    break
        print("-------------------")
        print("Token: ", next_token)
        url = create_url(keyword, start_list[i],end_list[i], max_results)
        json_response = connect_to_endpoint(url[0], headers, url[1], next_token)
        result_count = json_response['meta']['result_count']

        if 'next_token' in json_response['meta']:
            # Save the token to use for next call
            next_token = json_response['meta']['next_token']
            print("Next Token: ", next_token)
            if result_count is not None and result_count > 0 and next_token is not None:
                print("Start Date: ", start_list[i])
                append_to_csv(json_response, "data_texas.csv")
                count += result_count
                total_tweets += result_count
                print("Total # of Tweets added: ", total_tweets)
                #print("data.csv")
                print("-------------------")
                time.sleep(5)                
        # If no next token exists
        else:
            if result_count is not None and result_count > 0:
                print("-------------------")
                print("Start Date: ", start_list[i])
                append_to_csv(json_response, "data_texas.csv")
                #print('data_texas.csv')
                count += result_count
                total_tweets += result_count
                print("Total # of Tweets added: ", total_tweets)
                print("-------------------")
                time.sleep(5)
            
            #Since this is the final request, turn flag to false to move to the next time period.
            flag = False
            next_token = None
        time.sleep(5)
print("Total number of results: ", total_tweets)

-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fosntce1g31vgzh72kbmmgzetfubgd
Start Date:  2021-02-10T00:00:00.000Z
# of Tweets added from this response:  485
Total # of Tweets added:  485
-------------------
-------------------
Token:  b26v89c19zqg8o3fosntce1g31vgzh72kbmmgzetfubgd
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fosntce1g1kqnd9xfw1oqp91270bct
Start Date:  2021-02-10T00:00:00.000Z
# of Tweets added from this response:  494
Total # of Tweets added:  979
-------------------
-------------------
Token:  b26v89c19zqg8o3fosntce1g1kqnd9xfw1oqp91270bct
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fosntce1g1jw0ie834imfecux9alj1
Start Date:  2021-02-10T00:00:00.000Z
# of Tweets added from this response:  488
Total # of Tweets added:  1467
-------------------
-------------------
Token:  b26v89c19zqg8o3fosntce1g1jw0ie834imfecux9alj1
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fosntce15mt6tbs5hhhnyvpiyvvp19
Sta

In [ ]:
df_texas = pd.read_csv(open('data_texas.csv'))
df_texas.head()

#saving the file to google drive
df_texas.to_csv('/content/drive/My Drive/texas_data.csv')
!cp data.csv "drive/My Drive/"